In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.special import softmax
#from pyibs import IBS
from pybads import BADS

import sys
sys.path.append('..')
from utils.env_fn import *
from utils.model import *
from utils.myibs import IBS

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Run IBS + BADS using a simple example 

In [2]:
# the real policy 
def real_policy(theta, s):
    p = np.array([[1/2, 1/6, 1/3],
                  [1/4, 1/2, 1/4]])
    return softmax(theta*p[s])

def real_nll(theta, s, r):
    p = np.array([[1/2, 1/6, 1/3],
                  [1/4, 1/2, 1/4]])
    probs = np.array([softmax(theta*p[i])[j] for i, j in zip(s, r)])
    return -np.sum(np.log(probs))

# define the generator
def policy_generator(theta, s):
    p = np.array([[1/2, 1/6, 1/3],
                  [1/4, 1/2, 1/4]])
    probs = np.array([softmax(theta*p[i]) for i in s])
    # Sample a choice for each row based on its probability distribution
    choices = np.array([np.random.choice(range(3), p=prob) for prob in probs])
    return choices

In [3]:
# generate a list of input 
orig_theta = 4
s = np.random.randint(0, 2, size=100)
r = policy_generator(orig_theta, s)
print('stimulus (first 10):', s[:10])
print('response (first 10):', r[:10])

stimulus (first 10): [0 1 0 0 1 0 0 1 0 1]
response (first 10): [0 2 0 1 2 2 1 0 1 1]


In [6]:
# estimate the nll using IBS
ibs = IBS(
    policy_generator,
    response_matrix=r, 
    design_matrix=s,
    vectorized=True,
    max_iter=10000,
    max_time=300,
)
for theta in [.1, .5, 1, 2, 4, 8]:
    nll_real = real_nll(theta, s, r)
    nll_est = ibs(theta, num_reps=20)
    print(f'theta={theta}: real_nll={nll_real:.4f}, est_nll={nll_est:.4f}')

theta=0.1: real_nll=109.2777, est_nll=106.7908
theta=0.5: real_nll=107.1076, est_nll=107.9549
theta=1: real_nll=104.7669, est_nll=102.5443
theta=2: real_nll=101.3330, est_nll=103.3072
theta=4: real_nll=99.3659, est_nll=98.7399
theta=8: real_nll=112.4478, est_nll=110.6434


In [8]:
# fit parameter using IBS
target = lambda theta: ibs(theta, num_reps=20)

# fit model using bads
lb = np.array([0])
ub = np.array([20])
plb = np.array([.1])
pub = np.array([5])
theta0 = plb + np.random.rand(1) * (pub - plb)
bads_opt = {
    'uncertainty_handling': True,
    'noise_final_samples': 0,
    'display': 'off',
}
bads_ibs = BADS(
    target,
    theta0,
    lb,
    ub,
    plb,
    pub,
    options=bads_opt
)

theta_ibs = bads_ibs.optimize()
print('original theta:', orig_theta)
print('estimated theta:', theta_ibs['x'])

original theta: 4
estimated theta: [3.82595655]
